In [1]:
%load_ext autoreload
%autoreload 2

import torch
import torchvision
import torch.nn.functional as F
from torch import nn
from sklearn.metrics import precision_recall_fscore_support
import numpy as np

# manage ray's relative imports
import ray
runtime_env = {"working_dir": ".." }
ray.init(runtime_env=runtime_env, dashboard_port=13065, include_dashboard=True)

from ray import tune
from ray.tune.suggest.optuna import OptunaSearch
from ray.tune import JupyterNotebookReporter

# manage beams's relative imports
import sys
sys.path.append('..')

from src.beam import beam_arguments, Experiment, Study
from src.beam import UniversalDataset, UniversalBatchSampler
from src.beam import Algorithm
from src.beam import LinearNet
from torchvision import transforms
import matplotlib.pyplot as plt

from src.beam import DataTensor
from src.beam.utils import is_notebook
from cifar10_example import cifar10_algorithm_generator, Cifar10Network

from ray.tune.suggest.hebo import HEBOSearch

2022-06-20 09:11:49,276	INFO services.py:1456 -- View the Ray dashboard at http://127.0.0.1:13065
2022-06-20 09:11:49,781	INFO packaging.py:388 -- Creating a file package for local directory '..'.
2022-06-20 09:11:49,805	INFO packaging.py:241 -- Pushing file package 'gcs://_ray_pkg_bd2a01d656db6bf1.zip' (0.87MiB) to Ray cluster...
2022-06-20 09:11:49,819	INFO packaging.py:243 -- Successfully pushed file package 'gcs://_ray_pkg_bd2a01d656db6bf1.zip'.


In [2]:
# path_to_data = '/localdata/elads/data/datasets/cifar10'
# root_dir = '/localdata/elads/data/cifar10'

path_to_data = '/home/shared/data/dataset/cifar10'
root_dir = '/home/shared/data/results/cifar10'

## Training with a single worker

In [7]:
# here you put all actions which are performed only once before initializing the workers
# for example, setti`ng running arguments and experiment:

args = beam_arguments(f"--project-name=cifar10 --root-dir={root_dir} --algorithm=CIFAR10Algorithm --device=1 --half --lr-d=1e-4 --batch-size=512",
                      "--n-epochs=40 --epoch-length-train=50000 --epoch-length-eval=10000 --clip=0 --parallel=1 --accumulate=1 --cudnn-benchmark",
                      "--weight-decay=.00256 --beta1=0.9 --beta2=0.9", 
                      path_to_data=path_to_data, gamma=1., dropout=.0, activation='celu', channels=512,
                      scale_down=.7, scale_up=1.4, ratio_down=.7, ratio_up=1.4)

experiment = Experiment(args)

2022-06-20 08:42:51.854 | INFO     | src.beam.experiment:__init__:186 - beam project: cifar10
2022-06-20 08:42:51.856 | INFO     | src.beam.experiment:__init__:187 - Experiment Hyperparameters
2022-06-20 08:42:51.857 | INFO     | src.beam.experiment:__init__:192 - project_name: cifar10
2022-06-20 08:42:51.860 | INFO     | src.beam.experiment:__init__:192 - identifier: debug
2022-06-20 08:42:51.861 | INFO     | src.beam.experiment:__init__:192 - algorithm: CIFAR10Algorithm
2022-06-20 08:42:51.862 | INFO     | src.beam.experiment:__init__:192 - mp_port: None
2022-06-20 08:42:51.863 | INFO     | src.beam.experiment:__init__:192 - root_dir: /home/shared/data/results/cifar10
2022-06-20 08:42:51.864 | INFO     | src.beam.experiment:__init__:192 - reload: False
2022-06-20 08:42:51.866 | INFO     | src.beam.experiment:__init__:192 - resume: -1
2022-06-20 08:42:51.867 | INFO     | src.beam.experiment:__init__:192 - override: False
2022-06-20 08:42:51.867 | INFO     | src.beam.experiment:__init_

## Train with single or multiple workers

In [8]:
alg = experiment(cifar10_algorithm_generator)

2022-06-20 08:42:53.793 | INFO     | src.beam.experiment:run:554 - Single worker mode
2022-06-20 08:42:53.795 | INFO     | src.beam.experiment:run_worker:48 - Worker: 1/1 is running...


train:   0%|          | 0/96 [00:00<?, ?it/s]

0.08


test:   0%|          | 0/19 [00:00<?, ?it/s]


2022-06-20 08:42:57.555 | INFO     | src.beam.experiment:save_model_results:400 - Finished epoch 1/40:
2022-06-20 08:42:57.557 | INFO     | src.beam.experiment:log_data:446 - train:
2022-06-20 08:42:57.558 | INFO     | src.beam.experiment:log_data:450 - loss 854.714 	|
2022-06-20 08:42:57.559 | INFO     | src.beam.experiment:log_data:450 - acc 0.560872 	|
2022-06-20 08:42:57.559 | INFO     | src.beam.experiment:log_data:450 - lr 0.0001 	|
2022-06-20 08:42:57.560 | INFO     | src.beam.experiment:log_data:446 - test:
2022-06-20 08:42:57.561 | INFO     | src.beam.experiment:log_data:450 - loss 742.316 	|
2022-06-20 08:42:57.561 | INFO     | src.beam.experiment:log_data:450 - acc 0.693976 	|


...

train:   0%|          | 0/96 [00:00<?, ?it/s]

0.05714285714285716


test:   0%|          | 0/19 [00:00<?, ?it/s]


2022-06-20 08:45:16.361 | INFO     | src.beam.experiment:save_model_results:400 - Finished epoch 35/40:
2022-06-20 08:45:16.363 | INFO     | src.beam.experiment:log_data:446 - train:
2022-06-20 08:45:16.364 | INFO     | src.beam.experiment:log_data:450 - loss 454.206 	|
2022-06-20 08:45:16.365 | INFO     | src.beam.experiment:log_data:450 - acc 1 	|
2022-06-20 08:45:16.366 | INFO     | src.beam.experiment:log_data:450 - lr 6.85714e-06 	|
2022-06-20 08:45:16.367 | INFO     | src.beam.experiment:log_data:446 - test:
2022-06-20 08:45:16.368 | INFO     | src.beam.experiment:log_data:450 - loss 598.342 	|
2022-06-20 08:45:16.369 | INFO     | src.beam.experiment:log_data:450 - acc 0.871711 	|


KeyboardInterrupt: 

In [9]:
np.mean(alg.evaluate('test')['scalar']['acc'])

NameError: name 'alg' is not defined

In [6]:
# alg.optimizers['net'].scaler.get_scale()

## Hyperparameter search with native optuna

In [ ]:
args = beam_arguments(f"--project-name=cifar10 --root-dir={root_dir} --algorithm=CIFAR10Algorithm --device=1 --half --lr-d=1e-4 --batch-size=512",
                      "--n-epochs=40 --epoch-length-train=50000 --epoch-length-eval=10000 --clip=0 --parallel=1 --accumulate=1 --cudnn-benchmark",
                      "--weight-decay=.00256 --beta1=0.9 --beta2=0.9", 
                      path_to_data=path_to_data, gamma=1., dropout=.0, activation='celu', channels=512,
                      scale_down=.7, scale_up=1.4, ratio_down=.7, ratio_up=1.4)

study = Study(cifar10_algorithm_generator, args)

In [14]:
def suggest(trial):
    lr = trial.suggest_loguniform("lr", 1e-3, 2e-2)
    print('My suggestion')
    print(lr)
    return {'lr_dense': lr}
    

In [15]:
study.optuna(suggest, direction='maximize', n_jobs=1, n_trials=10)

[I 2022-06-20 08:46:21,221] A new study created in memory with name: cifar10/CIFAR10Algorithm/debug_hp_optimization_20220620_084618


My suggestion
0.009781018192999246
2022-06-20 08:46:21.224 | INFO     | src.beam.experiment:runner_optuna:114 - Next Hyperparameter suggestion:
2022-06-20 08:46:21.225 | INFO     | src.beam.experiment:runner_optuna:116 - lr_dense: 0.009781018192999246
2022-06-20 08:46:21.229 | INFO     | src.beam.experiment:__init__:262 - Creating new experiment
2022-06-20 08:46:21.229 | INFO     | src.beam.experiment:__init__:277 - Experiment directory is: /home/shared/data/results/cifar10/cifar10/CIFAR10Algorithm/debug_hp_optimization_20220620_084618/0000_20220620_084621
2022-06-20 08:46:21.239 | INFO     | src.beam.experiment:run:554 - Single worker mode
2022-06-20 08:46:21.240 | INFO     | src.beam.experiment:run_worker:48 - Worker: 1/1 is running...
0.08
0.16
0.24
0.32
0.4
0.38857142857142857
0.37714285714285717
0.3657142857142857
0.3542857142857143
0.34285714285714286
0.33142857142857146
0.32
0.3085714285714286


KeyboardInterrupt: 

## Hyperparameter search with ray-tune and optuna

In [3]:
args = beam_arguments(f"--project-name=cifar10 --root-dir={root_dir} --algorithm=CIFAR10Algorithm --device=1 --half --lr-d=1e-4 --batch-size=512",
                      "--n-epochs=40 --epoch-length-train=50000 --epoch-length-eval=10000 --clip=0 --parallel=1 --accumulate=1 --cudnn-benchmark",
                      "--weight-decay=.00256 --beta1=0.9 --beta2=0.9", 
                      path_to_data=path_to_data, gamma=1., dropout=.0, activation='celu', channels=512,
                      scale_down=.7, scale_up=1.4, ratio_down=.7, ratio_up=1.4)

study = Study(cifar10_algorithm_generator, args)

2022-06-20 09:13:31.344 | INFO     | src.beam.experiment:__init__:85 - Hyperparameter Optimization
2022-06-20 09:13:31.345 | INFO     | src.beam.experiment:__init__:86 - beam project: cifar10
2022-06-20 09:13:31.346 | INFO     | src.beam.experiment:__init__:87 - Experiment Hyperparameters
2022-06-20 09:13:31.348 | INFO     | src.beam.experiment:__init__:90 - project_name: cifar10
2022-06-20 09:13:31.350 | INFO     | src.beam.experiment:__init__:90 - identifier: debug_hp_optimization_20220620_091331
2022-06-20 09:13:31.351 | INFO     | src.beam.experiment:__init__:90 - algorithm: CIFAR10Algorithm
2022-06-20 09:13:31.352 | INFO     | src.beam.experiment:__init__:90 - mp_port: None
2022-06-20 09:13:31.353 | INFO     | src.beam.experiment:__init__:90 - root_dir: /home/shared/data/results/cifar10
2022-06-20 09:13:31.354 | INFO     | src.beam.experiment:__init__:90 - reload: False
2022-06-20 09:13:31.355 | INFO     | src.beam.experiment:__init__:90 - resume: -1
2022-06-20 09:13:31.356 | INFO

In [ ]:
# hebo = HEBOSearch(metric="mean_accuracy", mode="max")

# analysis = study.tune(config={"lr_dense": tune.loguniform(1e-3, 2e-2),
#                               "weight_decay": tune.loguniform(1e-6, 1e-4),
#                               "gamma": tune.loguniform(.1, .9),
#                               "dropout": tune.uniform(0, .75),
#                               "scale_down": tune.uniform(0.4, .7),
#                               "scale_up": tune.uniform(0.9, 1.2),
#                               "ratio_down": tune.uniform(0.7, .95),
#                               "ratio_up": tune.uniform(1.05, 1.4),
#                               "channels": tune.choice([128, 256, 512]),
#                               "batch_size": tune.choice([512, 1024, 2048]),},
#                        metric="mean_accuracy",
#                        max_concurrent_trials=4,
#                        resources_per_trial={"gpu": 1},
#                        mode="max",
#                        search_alg=hebo,
#                       progress_reporter=JupyterNotebookReporter(overwrite=True),
#                        num_samples=400)


analysis = study.tune(config={"lr_dense": tune.loguniform(1e-3, 2e-2),
                              "weight_decay": tune.loguniform(1e-6, 1e-4),
                              "gamma": tune.loguniform(.1, .9),
                              "dropout": tune.uniform(0, .75),
                              "scale_down": tune.uniform(0.4, .7),
                              "scale_up": tune.uniform(0.9, 1.5),
                              "ratio_down": tune.uniform(0.5, .95),
                              "ratio_up": tune.uniform(1.05, 1.5),
                              "channels": tune.choice([128, 256, 512]),
                              "batch_size": tune.choice([512, 1024, 2048]),
                              "activation": tune.choice(['relu', 'celu', 'gelu']),},
                       metric="mean_accuracy",
                       max_concurrent_trials=4,
                       resources_per_trial={"gpu": 2},
                       mode="max",
                       search_alg=OptunaSearch(),
                      progress_reporter=JupyterNotebookReporter(overwrite=True),
                       num_samples=400)


Trial name,status,loc,activation,batch_size,channels,dropout,gamma,lr_dense,ratio_down,ratio_up,scale_down,scale_up,weight_decay,acc,iter,total time (s)
runner_tune_0cd938f6,RUNNING,172.17.0.7:37347,gelu,2048,512,0.309856,0.244916,0.00461226,0.667457,1.19168,0.451295,1.39603,8.98847e-05,0.100586,40,113.829
runner_tune_0f93c30e,PENDING,,celu,2048,512,0.667971,0.355449,0.0129124,0.752898,1.37893,0.600333,0.95085,1.97476e-05,,,


Result for runner_tune_0cd938f6:
  date: 2022-06-20_09-21-11
  done: true
  experiment_id: 4ea3eae44790458a9de46bb1d7007127
  experiment_tag: 1_activation=gelu,batch_size=2048,channels=512,dropout=0.30986,gamma=0.24492,lr_dense=0.0046123,ratio_down=0.66746,ratio_up=1.1917,scale_down=0.4513,scale_up=1.396,weight_decay=8.9885e-05
  hostname: 56f38f239d73
  iterations_since_restore: 40
  mean_accuracy: 0.1005859375
  node_ip: 172.17.0.7
  pid: 37347
  time_since_restore: 113.82920527458191
  time_this_iter_s: 2.748023509979248
  time_total_s: 113.82920527458191
  timestamp: 1655716871
  timesteps_since_restore: 0
  training_iteration: 40
  trial_id: 0cd938f6
  warmup_time: 0.004626750946044922
  


2022-06-20 09:21:12,318	INFO trial_runner.py:803 -- starting runner_tune_0f93c30e


(runner_tune pid=37457) 2022-06-20 09:21:15.715 | INFO     | src.beam.experiment:__init__:262 - Creating new experiment
(runner_tune pid=37457) 2022-06-20 09:21:15.716 | INFO     | src.beam.experiment:__init__:277 - Experiment directory is: /home/shared/data/results/cifar10/cifar10/CIFAR10Algorithm/debug_hp_optimization_20220620_091331/0001_20220620_092115
(runner_tune pid=37457) 2022-06-20 09:21:15.721 | INFO     | src.beam.experiment:run:554 - Single worker mode
(runner_tune pid=37457) 2022-06-20 09:21:15.722 | INFO     | src.beam.experiment:run_worker:48 - Worker: 1/1 is running...
